In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

>  Análise Exploratória dos Dado

In [ ]:
df = pd.read_csv("../input/pima-indians-diabetes-database/diabetes.csv")
df.head()

In [ ]:
df.info()

In [ ]:
df.isna().any()

In [ ]:
df.info(verbose=True)

In [ ]:
print(df.dtypes)

In [ ]:
print(df.info())

In [ ]:
df.head(10)

In [ ]:
# Calcule o valor médio do IMC
median_bmi = df['BMI'].median()
# Substitua-o na coluna de IMC do
# conjunto de dados onde os valores são 0
df['BMI'] = df['BMI'].replace(
    to_replace=0, value=median_bmi)

median_bloodp = df['BloodPressure'].median()
# Substitua-o na coluna BloodP do
# conjunto de dados onde os valores são 0
df['BloodPressure'] = df['BloodPressure'].replace(
    to_replace=0, value=median_bloodp)

# Calcule o valor mediano para PlGlcConc
median_plglcconc = df['Glucose'].median()
# Substitua-o na coluna PlGlcConc do
# conjunto de dados onde os valores são 0
df['Glucose'] = df['Glucose'].replace(
    to_replace=0, value=median_plglcconc)

# Calcule o valor médio para SkinThick
median_skinthick = df['SkinThickness'].median()
# Substitua-o na coluna SkinThick do
# conjunto de dados onde os valores são 0
df['SkinThickness'] = df['SkinThickness'].replace(
    to_replace=0, value=median_skinthick)

# Calcule o valor médio para SkinThick
median_skinthick = df['Insulin'].median()
# Substitua-o na coluna SkinThick do
# conjunto de dados onde os valores são 0
df['Insulin'] = df['Insulin'].replace(
    to_replace=0, value=median_skinthick)

In [ ]:
df['Outcome'].value_counts().plot.bar();

> Feature Engineering¶¶


In [ ]:
def set_bmi(row):
    if row["BMI"] < 18.5:
        return "Under"
    elif row["BMI"] >= 18.5 and row["BMI"] <= 24.9:
        return "Healthy"
    elif row["BMI"] >= 25 and row["BMI"] <= 29.9:
        return "Over"
    elif row["BMI"] >= 30:
        return "Obese"

In [ ]:
df = df.assign(BM_DESC=df.apply(set_bmi, axis=1))

In [ ]:
df.head()

Característica 2: Faixa indicativa de insulina Se o nível de insulina (insulina sérica de 2 horas (mu U / ml)) for> = 16 e <= 166, então é uma faixa normal, caso contrário, é considerada anormal

In [ ]:
def set_insulin(row):
    if row["Insulin"] >= 16 and row["Insulin"] <= 166:
        return "Normal"
    else:
        return "Abnormal"

In [ ]:
df = df.assign(INSULIN_DESC=df.apply(set_insulin, axis=1))

df.head()

Característica de segregação e variável de destino¶


In [ ]:
X=pd.get_dummies(df)
cols_drop=['Outcome','BM_DESC_Under']
X=X.drop(cols_drop,axis=1)

y = df['Outcome']

In [ ]:
X.head()

> Divisão de teste de trem¶


In [ ]:
y.head()


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,stratify=y, random_state = 1234)


Normalização de Dados

Usei a normalização Z-score. Os escores Z são valores de dados transformados linearmente com uma média de zero e um desvio padrão de 1. Os escores Z também são conhecidos como escores padronizados; são pontuações (ou valores de dados) que receberam um padrão comum.

Se a média populacional e o desvio padrão populacional forem conhecidos, a pontuação padrão de uma pontuação bruta x [1] é calculada como

In [ ]:
min_train = X_train.min()
range_train = (X_train - min_train).max()
X_train = (X_train - min_train)/range_train

In [ ]:
min_test = X_test.min()
range_test = (X_test - min_test).max()
X_test = (X_test - min_test)/range_test

In [ ]:
X_test.isna().sum()

Modelo teste


Random Forest:¶

A random forest is a meta estimator that fits a number of decision tree classifiers on various sub-samples of the dataset and uses averaging to improve the predictive accuracy and control over-fitting.RandomForestClassifie

In [ ]:
from sklearn.ensemble import RandomForestClassifier


In [ ]:
Model=RandomForestClassifier(max_depth=2)
Model.fit(X_train,y_train)
y_pred=Model.predict(X_test)
print(classification_report(y_pred, y_test))
print(confusion_matrix(y_pred,y_test))


In [ ]:
#Pontuação de precisão
print('accuracy is ',accuracy_score(y_pred,y_test))

In [ ]:
# Fazendo a matriz de confusão e o relatório de classificação
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(data =  cm , annot = True , cmap = "Blues" , linewidths= 1 , linecolor= "black")

report = classification_report(y_test , y_pred)
print(report)

**Conclui-se**

Uma floresta aleatória é um metaestimador que ajusta uma série de classificadores de árvore de decisão em várias subamostras do conjunto de dados e usa média para melhorar a a precisão da previsão para controlar o sobreajuste.RandomForestClassifie. O tamanho da subamostra é sempre o mesmo tal qual o tamanho da amostra de entrada original.Assim Matriz de Confusão, uma ferramenta muito usada para avaliações de modelos de classificação em Aprendizado de Máquina (mais comumente chamado de Machine Learning).Para termos dados de exemplo, imagine que eu tivesse um modelo que, dada as características fisiológicas de uma paciente, dissesse que a mesma está em diabetes ou não. Porém, para o nosso contexto, eu só trarei uma lista com os dados reais, e uma lista de dados preditos pelo modelo, ambos dados fake(falsos).